In [1]:
# Import needed libraries
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import glob
import os

In [2]:
def readImages(images_path):
    """
    readImages creates an array of images to be worked on.
    Args:
        images_path: path to where images are stored, not forgetting image format
        example: ../../*.bmp or ../../*.jpg, * indicating any filename with the following format
    """  
    img_array = []
    print('Reading images ...')
    for filename in glob.glob(images_path):
        img = cv.imread(filename)
        img_array.append(img)
    print('Images read successfully.')
    return img_array

In [3]:
def reflectionRemover(images_array, threshold_value = 20, reflection_size = 250):
    """
    reflectionRemover removes reflections from PIV images that affects results of work.
    Args:
        images_array: an array of all PIV images to be worked on
        threshold_value: threshold value for blob detection and removal, default value is 20
        reflection_size: size of blob in terms of pixels to be removed, default value is 250 
    
    """
    print("Removing reflections ...")
    results = []
    for img in images_array:
        # Convert image to grayscale
        frame_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

        # Extract blobs
        ret,thresh = cv.threshold(frame_gray,threshold_value,255,cv.THRESH_BINARY) # distinguishing the pixel intensity
        contours, hierarchy = cv.findContours(thresh,cv.RETR_EXTERNAL,cv.CHAIN_APPROX_NONE)

        # Distinguish bubbles and reflections
        bubbles = []
        for keyvector in contours:
            if keyvector.size > reflection_size:
                bubbles.append(keyvector)

        # Remove bubbles and reflections
        image = img.copy()
        results_image = cv.drawContours(image, bubbles, -1, (0,0,0), -1)

        # Store results
        results.append(results_image)

    print("Reflection removal completed successfuly.")
    return results



In [5]:
def folder(name):
    """
    This function creates folders and takes foldername as input.
    Args:
        name: User defined folder 
        
    """
    try:
        if not os.path.exists(name):
            os.mkdir(name)
    except OSError:
        print('Error: Creating directory of ', name)
    return

In [6]:
def resultSaver(results_array, foldername):
    """
    resultSaver is used to save an array of images onto to the computer in a folder named Output.

    Args:
        results_array: this is an array of the resulting images after reflection removal
    
    """
    print("Saving resulting images to Output folder ...")
    counter = 0
    for result in results_array:
        file_name = foldername + "/Output" + str(counter) + ".jpg"
        cv.imwrite(file_name, result)
        counter += 1
    print("Saving resulting images to Output folder completed successfuly.")
    return

In [7]:
# Creat Output folder with user defined name
foldername = "PIV"
folder(foldername)

In [8]:
# read images into one array 
img_array = readImages("*.bmp")

Reading images ...
Images read successfully.


In [9]:
# Reflection removal
res_array = reflectionRemover(img_array, 30, 280)

Removing reflections ...
Reflection removal completed successfuly.


In [10]:
# Save resulting images
resultSaver(res_array, foldername)

Saving resulting images to Output folder ...
Saving resulting images to Output folder completed successfuly.
